In [2]:
import numpy as np
import pandas as pd
from pandas.io import gbq

import pickle

from pitched_recommend import Recommender

In [4]:
playlists_sql = "SELECT * FROM `umg-comm-tech-dev.browse_recommender.target_browse_playlists`"
playlists_list = gbq.read_gbq(playlists_sql, dialect='standard', project_id = 'umg-comm-tech-dev')

playlists_data_sql = "SELECT * FROM `umg-comm-tech-dev.browse_recommender.playlist_data`"
playlists_data = gbq.read_gbq(playlists_data_sql, dialect='standard', project_id = 'umg-comm-tech-dev')
playlists_data['long_uri'] = ['spotify:user:'+str(x) for x in playlists_data.playlist_uri]

data = playlists_list.merge(playlists_data, how='left', left_on='playlist_uri', right_on='long_uri')

In [5]:
data.head()

,territory,category_id,Position,playlist_uri_x,playlist_name_x,playlist_name_y,listeners,country_owner,playlist_owner,ranking,pl_ranking,playlist_uri_y,long_uri
0,SE,hiphop,1,spotify:user:spotify:playlist:37i9dQZF1DWXfgo3...,100,100,20932.0,SE,spotify,1.0,1.0,spotify:playlist:37i9dQZF1DWXfgo3OOonqa,spotify:user:spotify:playlist:37i9dQZF1DWXfgo3...
1,SE,toplists,4,spotify:user:spotify:playlist:37i9dQZF1DWXfgo3...,100,100,20932.0,SE,spotify,1.0,1.0,spotify:playlist:37i9dQZF1DWXfgo3OOonqa,spotify:user:spotify:playlist:37i9dQZF1DWXfgo3...
2,BR,popculture,2,spotify:user:spotify:playlist:37i9dQZF1DWWoqQt...,188,188,3418.0,BR,spotify,1.0,1.0,spotify:playlist:37i9dQZF1DWWoqQt9ZUTJC,spotify:user:spotify:playlist:37i9dQZF1DWWoqQt...
3,CO,chill,63,spotify:user:spotify:playlist:37i9dQZF1DX6KOz8...,9PM,9PM,11138.0,MX,spotify,1.0,1.0,spotify:playlist:37i9dQZF1DX6KOz8PNkOlS,spotify:user:spotify:playlist:37i9dQZF1DX6KOz8...
4,ES,chill,72,spotify:user:spotify:playlist:37i9dQZF1DX6KOz8...,9PM,9PM,11138.0,MX,spotify,1.0,1.0,spotify:playlist:37i9dQZF1DX6KOz8PNkOlS,spotify:user:spotify:playlist:37i9dQZF1DX6KOz8...


In [6]:
### Get the 'same_as_country' column

data['same_as_country'] = [1 if x==y else 0 for (x,y) in zip(data.territory,data.country_owner)]

In [10]:
### Get the 'count_appearance' column

playlist_count = data.playlist_uri_x.value_counts()
playlist_count = pd.DataFrame(playlist_count).reset_index().rename(columns={'index':'playlist_uri',
                                                                            'playlist_uri_x':'count_appearance'})
data = data.merge(playlist_count, how='left', left_on = 'playlist_uri_x', right_on = 'playlist_uri')
#data.head()

,territory,category_id,Position,playlist_uri_x,playlist_name_x,playlist_name_y,listeners,country_owner,playlist_owner,ranking,pl_ranking,playlist_uri_y,long_uri,same_as_country,playlist_uri,count_appearance
0,SE,hiphop,1,spotify:user:spotify:playlist:37i9dQZF1DWXfgo3...,100,100,20932.0,SE,spotify,1.0,1.0,spotify:playlist:37i9dQZF1DWXfgo3OOonqa,spotify:user:spotify:playlist:37i9dQZF1DWXfgo3...,1,spotify:user:spotify:playlist:37i9dQZF1DWXfgo3...,2
1,SE,toplists,4,spotify:user:spotify:playlist:37i9dQZF1DWXfgo3...,100,100,20932.0,SE,spotify,1.0,1.0,spotify:playlist:37i9dQZF1DWXfgo3OOonqa,spotify:user:spotify:playlist:37i9dQZF1DWXfgo3...,1,spotify:user:spotify:playlist:37i9dQZF1DWXfgo3...,2
2,BR,popculture,2,spotify:user:spotify:playlist:37i9dQZF1DWWoqQt...,188,188,3418.0,BR,spotify,1.0,1.0,spotify:playlist:37i9dQZF1DWWoqQt9ZUTJC,spotify:user:spotify:playlist:37i9dQZF1DWWoqQt...,1,spotify:user:spotify:playlist:37i9dQZF1DWWoqQt...,1
3,CO,chill,63,spotify:user:spotify:playlist:37i9dQZF1DX6KOz8...,9PM,9PM,11138.0,MX,spotify,1.0,1.0,spotify:playlist:37i9dQZF1DX6KOz8PNkOlS,spotify:user:spotify:playlist:37i9dQZF1DX6KOz8...,0,spotify:user:spotify:playlist:37i9dQZF1DX6KOz8...,13
4,ES,chill,72,spotify:user:spotify:playlist:37i9dQZF1DX6KOz8...,9PM,9PM,11138.0,MX,spotify,1.0,1.0,spotify:playlist:37i9dQZF1DX6KOz8PNkOlS,spotify:user:spotify:playlist:37i9dQZF1DX6KOz8...,0,spotify:user:spotify:playlist:37i9dQZF1DX6KOz8...,13


In [11]:
### Get the 'local' column

data['local'] = [1 if x<20 else 0 for x in data.count_appearance]

In [13]:
### Get the 'mean_listeners_cat_terr' column

by_cat_terr = data.groupby(by=['territory','category_id']).listeners.mean()
df_cat_terr = pd.DataFrame(by_cat_terr).reset_index()
df_cat_terr = df_cat_terr.rename(columns = {'listeners':'mean_listeners_cat_terr'})

data = data.merge(df_cat_terr, how='left', left_on = ['territory','category_id'], 
                  right_on = ['territory','category_id'])

In [15]:
### Get the 'popular' column

data['popular'] = [1 if x>y else 0 for (x,y) in zip(data.listeners,data.mean_listeners_cat_terr)]

In [17]:
### Get the 'score' column with the overall score
data['score'] = data.same_as_country + data.local + data.popular

In [18]:
cols = ['territory', 'category_id', 'Position', 'playlist_uri',
       'playlist_name_x', 'listeners', 'country_owner', 'count_appearance',
        'mean_listeners_cat_terr', 'same_as_country', 'local', 'popular',
        'score']

In [19]:
data = data[cols].sort_values(by=['territory','category_id','score'], ascending=False)
data_final = pd.DataFrame(data.groupby(by=['territory','category_id']).head(10).reset_index())

In [24]:
data_final.head()

,index,territory,category_id,Position,playlist_uri,playlist_name_x,listeners,country_owner,count_appearance,mean_listeners_cat_terr,same_as_country,local,popular,score
0,100,UY,workout,8,spotify:user:spotify:playlist:37i9dQZF1DX4eRPd...,Hype,452955.0,US,58,87462.893617,0,0,1,1
1,877,UY,workout,7,spotify:user:spotify:playlist:37i9dQZF1DWSJHnP...,Cardio,89598.0,US,58,87462.893617,0,0,1,1
2,2398,UY,workout,5,spotify:user:spotify:playlist:37i9dQZF1DX70RN3...,Workout,444881.0,US,58,87462.893617,0,0,1,1
3,8276,UY,workout,2,spotify:user:spotify:playlist:37i9dQZF1DX76Wlf...,Beast Mode,1343212.0,US,58,87462.893617,0,0,1,1
4,29452,UY,workout,1,spotify:user:spotify:playlist:37i9dQZF1DX9ZKyQ...,Para Entrenar,65602.0,MX,17,87462.893617,0,1,0,1


In [25]:
data_final.to_csv('processed_recs_for_andy_tab.csv', sep='\t', index=False)

In [21]:
def install_and_import(package):
    import importlib
    try:
        importlib.import_module(package)
    except ImportError:
        import pip
        pip.main(['install', package])
    finally:
        globals()[package] = importlib.import_module(package)



In [23]:
install_and_import('scipy.sparse')

In [35]:
test = pd.read_csv('/Users/daria/Downloads/recommender%2Fbrowse_recommendations_20181205 (1).csv', sep='\t', 
                   index_col=False)

In [36]:
test.shape

(299693, 7)

In [2]:
test2 = pd.read_csv('/Users/daria/development/umg/recommender/playlist_pitching/preprocessed_recs_for_andy_tab.csv',
                    sep='\t', 
                   index_col=False)

In [3]:
test2.shape

(102950, 7)

In [4]:
test2.head()

,territory,category,scores,isrc,seed_playlist_uri,track_artist,track_title
0,UY,workout,1.437311,USUG11800877,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Jay Rock,WIN
1,UY,workout,1.408097,USUM71800022,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Eminem,Chloraseptic (feat. 2 Chainz & Phresher) - Remix
2,UY,workout,1.347921,GBUM71028043,spotify:user:spotify:playlist:37i9dQZF1DXadOVC...,Ellie Goulding,Lights - Single Version
3,UY,workout,1.346981,USAT21704833,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Meek Mill,"That's My N**** (with Meek Mill, YG & Snoop Dogg)"
4,UY,workout,1.294945,USAT21101071,spotify:user:spotify:playlist:37i9dQZF1DXadOVC...,Gym Class Heroes,Stereo Hearts (feat. Adam Levine) - feat. Adam...


In [5]:
test2.category.unique()

array(['workout', 'travel', 'toplists', 'soul', 'sleep', 'roots',
       'romance', 'rock', 'rnb', 'reggae', 'punk', 'popculture', 'pop',
       'mood', 'metal', 'kpop', 'jazz', 'inspirational', 'indie_alt',
       'holidays', 'hiphop', 'gaming', 'funk', 'focus', 'family',
       'edm_dance', 'dinner', 'desi', 'decades', 'country', 'comedy',
       'classical', 'chill', 'blues', 'afro', 'word', 'sessions', 'party',
       'latin', 'francophone'], dtype=object)

In [39]:
test.category.unique()

array(['workout', 'travel', 'toplists', 'soul', 'sleep', 'roots',
       'romance', 'rock', 'rnb', 'reggae', 'punk', 'pop', 'mood', 'metal',
       'kpop', 'kids', 'jazz', 'inspirational', 'indie_alt', 'holidays',
       'hiphop', 'gaming', 'funk', 'focus', 'edm_dance', 'decades',
       'country', 'comedy', 'classical', 'chill', 'brazilian', 'blues',
       'arab', 'afro', 'party', 'latin', '2018'], dtype=object)

In [29]:
test.head()

,territory,category,scores,isrc,seed_playlist_uri,track_artist,track_title
0,AD,workout,1.437311,USUG11800877,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Jay Rock,WIN
1,AD,workout,1.408098,USUM71800022,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Eminem,Chloraseptic (feat. 2 Chainz & Phresher) - Remix
2,AD,workout,1.381630,USUM71710087,spotify:user:spotify:playlist:37i9dQZF1DWU4orV...,Post Malone,rockstar
3,AD,workout,1.339208,QMUY41600121,spotify:user:spotify:playlist:37i9dQZF1DX4FJyI...,Major Lazer,Run Up (feat. PARTYNEXTDOOR & Nicki Minaj)
4,AD,workout,1.320847,USLR50000417,spotify:user:spotify:playlist:37i9dQZF1DX76t63...,M.O.P.,Ante Up (Robbin Hoodz Theory)


In [3]:
with open('metadata_dict.pkl', 'rb') as f:
    metadata = pickle.load(f)

In [4]:
with open('/Users/daria/Downloads/saved_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [5]:
playlists = pd.read_csv('/Users/daria/Downloads/data_browse_playlists.csv', index_col='index')
playlists.head()

,territory,category_id,Position,playlist_uri,playlist_name_x,listeners,country_owner,count_appearance,mean_listeners_cat_terr,same_as_country,local,popular,score
index,,,,,,,,,,,,,
55589,UY,workout,10,spotify:user:spotify:playlist:37i9dQZF1DX4eRPd...,Hype,666033.0,US,58,108837.659574,0,0,1,1
55741,UY,workout,7,spotify:user:spotify:playlist:37i9dQZF1DX70RN3...,Workout,307011.0,US,58,108837.659574,0,0,1,1
55917,UY,workout,2,spotify:user:spotify:playlist:37i9dQZF1DX76Wlf...,Beast Mode,1460332.0,US,58,108837.659574,0,0,1,1
56575,UY,workout,1,spotify:user:spotify:playlist:37i9dQZF1DX9ZKyQ...,Para Entrenar,64519.0,MX,17,108837.659574,0,1,0,1
56597,UY,workout,11,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Power Workout,616480.0,US,57,108837.659574,0,0,1,1


In [6]:
all_territories = list(playlists.territory.unique())

rec = []

df=playlists.copy()

for territory in all_territories:
    #print("Producing recommendations for ", territory)
    terr_cat = list(df[df.territory==territory].category_id.unique())
    for category in terr_cat:
        #print("Producing recommendations for category ", category)
        terr_cat_playlists = list(df[(df.territory==territory)&(df.category_id==category)].playlist_uri.unique())
        rec.extend(model.rec_multiple_playlists(terr_cat_playlists, model.playlists_sparse, territory=territory,
                                          category = category, N=150))

No recommendations for territory-category  US popculture


KeyboardInterrupt: 

In [16]:
rec

[(17989,
  1.4373112,
  'spotify:user:spotify:playlist:37i9dQZF1DWUVpAXiEPK8P',
  'UY',
  'workout'),
 (72958,
  1.4080974,
  'spotify:user:spotify:playlist:37i9dQZF1DWUVpAXiEPK8P',
  'UY',
  'workout'),
 (14247,
  1.3479207,
  'spotify:user:spotify:playlist:37i9dQZF1DXadOVCgGhS7j',
  'UY',
  'workout'),
 (4351,
  1.3469807,
  'spotify:user:spotify:playlist:37i9dQZF1DWUVpAXiEPK8P',
  'UY',
  'workout'),
 (277,
  1.2949445,
  'spotify:user:spotify:playlist:37i9dQZF1DXadOVCgGhS7j',
  'UY',
  'workout'),
 (7716,
  1.2908813,
  'spotify:user:spotify:playlist:37i9dQZF1DX9ZKyQHcEFXZ',
  'UY',
  'workout'),
 (50763,
  1.2787248,
  'spotify:user:spotify:playlist:37i9dQZF1DX0HRj9P7NxeE',
  'UY',
  'workout'),
 (76578,
  1.2733085,
  'spotify:user:spotify:playlist:37i9dQZF1DWUVpAXiEPK8P',
  'UY',
  'workout'),
 (61311,
  1.2670068,
  'spotify:user:spotify:playlist:37i9dQZF1DXadOVCgGhS7j',
  'UY',
  'workout'),
 (29540,
  1.2645034,
  'spotify:user:spotify:playlist:37i9dQZF1DXadOVCgGhS7j',
  'UY'

In [19]:
model.isrcs[17989]

'USUG11800877'

In [14]:
recommendations

[('USUG11800877',
  1.4373111724853516,
  'spotify:user:spotify:playlist:37i9dQZF1DWUVpAXiEPK8P'),
 ('USUM71800022',
  1.4080973863601685,
  'spotify:user:spotify:playlist:37i9dQZF1DWUVpAXiEPK8P'),
 ('GBUM71028043',
  1.3479206562042236,
  'spotify:user:spotify:playlist:37i9dQZF1DXadOVCgGhS7j'),
 ('USAT21704833',
  1.3469806909561157,
  'spotify:user:spotify:playlist:37i9dQZF1DWUVpAXiEPK8P'),
 ('USAT21101071',
  1.2949445247650146,
  'spotify:user:spotify:playlist:37i9dQZF1DXadOVCgGhS7j'),
 ('CYA111500132',
  1.2908812761306763,
  'spotify:user:spotify:playlist:37i9dQZF1DX9ZKyQHcEFXZ'),
 ('USAT21801428',
  1.2787247896194458,
  'spotify:user:spotify:playlist:37i9dQZF1DX0HRj9P7NxeE'),
 ('USSM11704822',
  1.273308515548706,
  'spotify:user:spotify:playlist:37i9dQZF1DWUVpAXiEPK8P'),
 ('USRC11301020',
  1.267006754875183,
  'spotify:user:spotify:playlist:37i9dQZF1DXadOVCgGhS7j'),
 ('GBCTA1400065',
  1.2645033597946167,
  'spotify:user:spotify:playlist:37i9dQZF1DXadOVCgGhS7j'),
 ('GBUM71705

In [22]:
recommendations = [(model.isrcs[x[0]],float(x[1]),x[2],x[3],x[4]) for x in rec] # turns the results into a list of tuples (item_index_isrc, score)
recs_final = [(x[0],x[1],metadata['track_artist'][x[0]], metadata['track_title'][x[0]], metadata['major_label'][x[0]],
               metadata['label_studio'][x[0]],
               metadata['original_release_date'][x[0]], metadata['genre_name'][x[0]], metadata['parent_genre_name'][x[0]], 
               x[2], x[3], x[4]) for x in list(recommendations)]

In [23]:
all_columns = ['isrc','score','artist','title','major_label','label_studio','original_release_date','genre_name',
               'parent_genre_name','seed_playlist_uri','territory','category']
recs_df_final = pd.DataFrame(recs_final, columns = all_columns)

In [27]:
recs_df_final[recs_df_final.major_label=='UMG']

,isrc,score,artist,title,major_label,label_studio,original_release_date,genre_name,parent_genre_name,seed_playlist_uri,territory,category
0,USUG11800877,1.437311,Jay Rock,WIN,UMG,Jay Rock (TDE/ISR) JV,2018-06-15 00:00:00,Hip Hop/Rap,Hip Hop/Rap,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,UY,workout
1,USUM71800022,1.408097,Eminem,Chloraseptic (feat. 2 Chainz & Phresher) - Remix,UMG,Aftermath New JV-Old School,2018-01-08 00:00:00,Hip Hop/Rap,Hip Hop/Rap,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,UY,workout
2,GBUM71028043,1.347921,Ellie Goulding,Lights - Single Version,UMG,Polydor,2012-10-05 00:00:00,Pop,Pop,spotify:user:spotify:playlist:37i9dQZF1DXadOVC...,UY,workout
5,CYA111500132,1.290881,Tiësto,The Right Song,UMG,Marketing Labs,2016-07-22 00:00:00,Dance,Dance,spotify:user:spotify:playlist:37i9dQZF1DX9ZKyQ...,UY,workout
10,GBUM71705418,1.262508,Giggs,Linguo,UMG,Universal-Island Records Ltd.,2017-10-06 00:00:00,Hip Hop/Rap,Hip Hop/Rap,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,UY,workout
11,USUM71312880,1.257473,Aloe Blacc,The Man,UMG,Interscope,2013-10-25 00:00:00,Pop,Pop,spotify:user:spotify:playlist:37i9dQZF1DXadOVC...,UY,workout
18,USUM71807355,1.236965,Pusha T,What Would Meek Do?,UMG,"Getting Out Our Dreams, Inc./Def Jam Recordings",2018-05-25 00:00:00,Hip Hop/Rap,Hip Hop/Rap,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,UY,workout
25,QMZSY1600083,1.224953,DJ Snake,Let Me Love You - Tiësto's AFTR:HRS Mix,UMG,Marketing Labs,2017-03-17 00:00:00,Dance,Dance,spotify:user:spotify:playlist:37i9dQZF1DX9ZKyQ...,UY,workout
28,USUM71714098,1.213350,ScHoolboy Q,X (with 2 Chainz & Saudi),UMG,Black Panther (TDE/DMG) PS,2018-02-09 00:00:00,Hip Hop/Rap,Hip Hop/Rap,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,UY,workout
33,USUM71001801,1.207360,Neon Trees,Animal,UMG,Mercury Records,2010-01-01 00:00:00,Alternative,Alternative,spotify:user:spotify:playlist:37i9dQZF1DXadOVC...,UY,workout


In [30]:
test = pd.read_csv('/Users/daria/Downloads/recommender_browse_recommendations_browse_recommendations_20190314.csv', 
                   delimiter="\t")
test.head()

,territory,category,score,isrc,seed_playlist_uri,artist,title
0,UY,workout,1.437311,USUG11800877,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Jay Rock,WIN
1,UY,workout,1.408098,USUM71800022,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Eminem,Chloraseptic (feat. 2 Chainz & Phresher) - Remix
2,UY,workout,1.347921,GBUM71028043,spotify:user:spotify:playlist:37i9dQZF1DXadOVC...,Ellie Goulding,Lights - Single Version
3,UY,workout,1.290881,CYA111500132,spotify:user:spotify:playlist:37i9dQZF1DX9ZKyQ...,Tiësto,The Right Song
4,UY,workout,1.262508,GBUM71705418,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Giggs,Linguo


In [31]:
test.category.unique()

array(['workout', 'travel', 'toplists', 'soul', 'sleep', 'romance',
       'rock', 'rnb', 'reggae', 'punk', 'popculture', 'pop', 'mood',
       'metal', 'kpop', 'jazz', 'inspirational', 'indie_alt', 'hiphop',
       'gaming', 'funk', 'focus', 'edm_dance', 'dinner', 'desi',
       'decades', 'comedy', 'classical', 'chill', 'blues', 'arab', 'afro',
       'sessions', 'party', 'ellen'], dtype=object)

In [33]:
test.territory.unique()

array(['UY', 'US', 'TW', 'TR', 'SV', 'SK', 'SG', 'SE', 'PY', 'PT', 'PL',
       'PH', 'PE', 'PA', 'NZ', 'NO', 'NL', 'NI', 'MY', 'MX', 'MT', 'MC',
       'LV', 'LU', 'LT', 'LI', 'IT', 'IS', 'IE', 'HU', 'HN', 'HK', 'GT',
       'GR', 'GB', 'FR', 'FI', 'ES', 'EE', 'EC', 'DO', 'DK', 'DE', 'CZ',
       'CY', 'CR', 'CO', 'CL', 'CH', 'CA', 'BR', 'BO', 'BG', 'BE', 'AU',
       'AT', 'AR', 'AD'], dtype=object)

In [37]:
test[(test.territory=='DE')&(test.category=='pop')][20:]

,territory,category,score,isrc,seed_playlist_uri,artist,title
58294,DE,pop,1.153650,USAT21704316,spotify:user:spotify:playlist:37i9dQZF1DX0gbcr...,Why Don't We,Trust Fund Baby
58295,DE,pop,1.153231,GBUM71700020,spotify:user:spotify:playlist:37i9dQZF1DX8ttEd...,Sigrid,Don't Kill My Vibe
58296,DE,pop,1.151701,USUM71800126,spotify:user:spotify:playlist:37i9dQZF1DX0gbcr...,Maroon 5,Wait (feat. A Boogie Wit da Hoodie)
58297,DE,pop,1.150706,USJMT1500001,spotify:user:spotify:playlist:37i9dQZF1DX274mI...,Rihanna,FourFiveSeconds
58298,DE,pop,1.146571,US43C1602374,spotify:user:spotify:playlist:37i9dQZF1DX8ttEd...,Welshly Arms,Legendary
58299,DE,pop,1.138815,DEE861800536,spotify:user:spotify:playlist:37i9dQZF1DX0gbcr...,Alle Farben,H.O.L.Y.
58300,DE,pop,1.138324,USUM71813582,spotify:user:spotify:playlist:37i9dQZF1DX0gbcr...,Shawn Mendes,Lost In Japan - Remix
58301,DE,pop,1.136379,DEUM71601962,spotify:user:spotify:playlist:37i9dQZF1DX0gbcr...,Dua Lipa,New Rules
58302,DE,pop,1.131816,USUM71800031,spotify:user:spotify:playlist:37i9dQZF1DX8ttEd...,Noah Kahan,Hurt Somebody (With Julia Michaels)
58303,DE,pop,1.124248,USUM71703088,spotify:user:spotify:playlist:37i9dQZF1DX0gbcr...,Kendrick Lamar,LOVE. FEAT. ZACARI.


In [40]:
meta_df = pd.DataFrame(metadata).reset_index()
meta_df.head()

,index,track_artist,track_title,major_label,label_studio,original_release_date,genre_name,parent_genre_name
0,01TRB0660177,Various Artists,Wangsta,Unknown,Unknown,Unknown,Unknown,Unknown
1,4GBSMU284405,Steven Jame$,Pack,Unknown,Unknown,Unknown,Unknown,Unknown
2,549571700187,Beataholics,Rio Brown Rap Beat Instrumental,Unknown,Unknown,Unknown,Unknown,Unknown
3,549571700189,Beataholics,Love Rap Beat Instrumental,Unknown,Unknown,Unknown,Unknown,Unknown
4,9555150769372,Khalil Fong,悟空,Unknown,Unknown,Unknown,Unknown,Unknown


In [43]:
test_meta = test.merge(meta_df, left_on='isrc', right_on='index')
test_meta.head()

,territory,category,score,isrc,seed_playlist_uri,artist,title,index,track_artist,track_title,major_label,label_studio,original_release_date,genre_name,parent_genre_name
0,UY,workout,1.437311,USUG11800877,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Jay Rock,WIN,USUG11800877,Jay Rock,WIN,UMG,Jay Rock (TDE/ISR) JV,2018-06-15 00:00:00,Hip Hop/Rap,Hip Hop/Rap
1,US,hiphop,1.261862,USUG11800877,spotify:user:spotify:playlist:37i9dQZF1DX1YPTA...,Jay Rock,WIN,USUG11800877,Jay Rock,WIN,UMG,Jay Rock (TDE/ISR) JV,2018-06-15 00:00:00,Hip Hop/Rap,Hip Hop/Rap
2,TW,workout,1.437311,USUG11800877,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Jay Rock,WIN,USUG11800877,Jay Rock,WIN,UMG,Jay Rock (TDE/ISR) JV,2018-06-15 00:00:00,Hip Hop/Rap,Hip Hop/Rap
3,TR,workout,1.437311,USUG11800877,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Jay Rock,WIN,USUG11800877,Jay Rock,WIN,UMG,Jay Rock (TDE/ISR) JV,2018-06-15 00:00:00,Hip Hop/Rap,Hip Hop/Rap
4,SV,workout,1.437311,USUG11800877,spotify:user:spotify:playlist:37i9dQZF1DWUVpAX...,Jay Rock,WIN,USUG11800877,Jay Rock,WIN,UMG,Jay Rock (TDE/ISR) JV,2018-06-15 00:00:00,Hip Hop/Rap,Hip Hop/Rap


In [44]:
test_meta.major_label.unique()

array(['UMG'], dtype=object)

In [45]:
model.playlists

['spotify:playlist:0000mSEZofZjMa5x6ooFMT',
 'spotify:playlist:000IDLrAdMwaTycyWNqphh',
 'spotify:playlist:000fE5txwhxHYyCiv8OpEm',
 'spotify:playlist:005HdHSl4fZSoF77oWLHBQ',
 'spotify:playlist:008KzFkiwx2CmmqfM2PYUG',
 'spotify:playlist:00BsmzJofCqjh2wAyxRLSN',
 'spotify:playlist:00GRU1Mzud6oZNRZ7K279j',
 'spotify:playlist:00GxL0MrfispCSnMB77M22',
 'spotify:playlist:00HFOyUKrEuJ35ZFU9wWOF',
 'spotify:playlist:00KH2xK7ojnGCST8Y18NVM',
 'spotify:playlist:00KSJjSBcsRqXx8HISmI7D',
 'spotify:playlist:00MjsUEn2rM8DTatRUDs73',
 'spotify:playlist:00N4I7XbtXisTJZZe1Mn8L',
 'spotify:playlist:00QS6EfayQE4LQNIOkAJk2',
 'spotify:playlist:00QlP9vJLw9c2yVITuDqSw',
 'spotify:playlist:00RzaBjM9vXGPWjoNktVPc',
 'spotify:playlist:00SAVe6tZGGszQrokRqO8w',
 'spotify:playlist:00Te9h6zm6nnohYHTbdqtb',
 'spotify:playlist:00UweE3RzaSf4i6ayIJxe0',
 'spotify:playlist:00WnyxQxvmiKCjTRqR6TMo',
 'spotify:playlist:00YvkOKwWpuxeOyKRGVOsW',
 'spotify:playlist:00ZpAMC09DdoN0lqIMjGKC',
 'spotify:playlist:00g0QmFaVDEXa